
# Step 1 - Preliminary data analysis
To get used to both MongoDB and the data at disposal, investigate first the collections and get used to the document and field stored in each.

* How many documents are present in each collection?
* Why the number of documents in PermanentParkings and PermanentBookings is similar?
* For which cities the system is collecting data?
* When the collection started? When the collection ended?
* What about the timezone of the timestamps?
Considering each city of your group, check
* How many cars are available in each city?
* How many bookings have been recorded on the December 2017 in each city?
* How many bookings have also the alternative transportation modes recorded in each city?

For each question, write the MongoDB query, and the answer you get. Add a brief comment, if useful, to justify the result that you get.

We'll used the python library **pymongo**

In [44]:
"""
    ICT in Transport System Laboratories:
    City assigned: 
    Torino
    Torino enjoy
    Portland
"""
from pymongo import MongoClient
from datetime import datetime
import time
import pprint #for a pretty print

"""Cluster configuration"""
url_cluster = "bigdatadb.polito.it"
port_number = 27017 #it's the default port in MongoCLient
authSource = 'carsharing' #the database to authenticate on. 
username = 'ictts'
psw = 'Ictts16!'

#client connection
client = MongoClient(host=url_cluster, port=port_number, ssl=True, authSource=authSource,
                     tlsAllowINvalidCertificates=True)

db = client['carsharing'] #the database we want to authenticate on.
db.authenticate(username, psw)

True

Now we are connected to the cluster and we have chosen the database on which we want to work.
The first question is: how many documents are present in each collection?
In MongoDB, documents are like rows and collections are like tables, in a RDBMS. So, it was asked to understand how many samples are present in each tables. First of all, we want to see what the collections are in the database: 

In [45]:
collections_ls = db.list_collection_names()
collections_ls

['enjoy_PermanentBookings',
 'test_collection',
 'PermanentBookings',
 'PermanentParkings',
 'system.indexes',
 'system.js',
 'enjoy_PermanentParkings',
 'ActiveBookings',
 'enjoy_DebugInfo',
 'ictts_PermanentBookings',
 'ictts_enjoy_PermanentBookings',
 'BigDive7_PermanentBookings',
 'enjoy_ActiveParkings',
 'DebugInfo',
 'ActiveParkings',
 'hexs_gen',
 'enjoy_ActiveBookings']

There is 17 collections in the database **carsharing**. We are only interested in some of these. In particular, we are interessed in:
* "ActiveBookings": Contains cars that are currently booked and not available
* "ActiveParkings": Contains cars that are currently parked and available
* “PermanentBookings": Contains all booking periods recorded so far
* "PermanentParkings": Contains all parking periods recorded so far

the same are available by enjoy company. Now, we want to know how many documents are in each of the collections that interest us. 


In [46]:
interest_collections = ['ActiveBookings', 'ActiveParkings', 'PermanentBookings', 'PermanentParkings',
                        'enjoy_ActiveBookings', 'enjoy_ActiveParkings', 
                        'enjoy_PermanentBookings', 'enjoy_PermanentParkings']
numberDocuments_dict = {}

for collection in interest_collections:
    numberDocuments_dict[collection] = db[collection].estimated_document_count()
    
numberDocuments_dict

{'ActiveBookings': 8743,
 'ActiveParkings': 4790,
 'PermanentBookings': 28180508,
 'PermanentParkings': 28312676,
 'enjoy_ActiveBookings': 0,
 'enjoy_ActiveParkings': 0,
 'enjoy_PermanentBookings': 6653472,
 'enjoy_PermanentParkings': 6689979}

In [47]:
cities = db['ActiveBookings'].distinct('city')
cities

['Amsterdam',
 'Austin',
 'Berlin',
 'Calgary',
 'Columbus',
 'Denver',
 'Firenze',
 'Frankfurt',
 'Hamburg',
 'Madrid',
 'Milano',
 'Montreal',
 'Munchen',
 'New York City',
 'Portland',
 'Rheinland',
 'Roma',
 'Seattle',
 'Stuttgart',
 'Torino',
 'Toronto',
 'Vancouver',
 'Washington DC',
 'Wien']

Now, we want to know what is the *start time* and the *end time* for each of the collection that interest us. First, let's see how a document is:

In [48]:
db.ActiveParkings.find_one()

{'_id': ObjectId('5a7087732ad85324efc00c6a'),
 'plate': 'BHF2074',
 'vendor': 'car2go',
 'final_time': -1,
 'loc': {'type': 'Point', 'coordinates': [-122.34949, 47.72745]},
 'init_time': 1517324135,
 'vin': 'WDCTG4GB1JJ426163',
 'smartPhoneRequired': True,
 'interior': 'GOOD',
 'exterior': 'GOOD',
 'address': '13524 Fremont Ave N, Seattle, WA 98133, USA',
 'init_date': datetime.datetime(2018, 1, 30, 6, 55, 35),
 'final_date': -1,
 'city': 'Seattle',
 'fuel': 100,
 'engineType': 'CE'}

In [ ]:
#i want to know if the start time is the same for all collections
a = list(db['ActiveBookings'].find().sort('init_time'))[0]
b = list(db['PermanentParkings'].find().sort('init_time'))[0]
a['init_time'], b['init_time']

In [ ]:
#it takes too many time 
startEnd_dict = {}
df_start = list(db['PermanentBookings'].find().sort('init_time'))[0]
df_end = list(db['PermanentBookings'].find().sort('init_time'))[-1]
startEnd_dict['PermanentBookings'] = {'start': datetime.fromtimestamp(df_start['init_time']), 'end': datetime.fromtimestamp(df_end['init_time'])}
startEnd_dict